In [1]:
# Import the necessary packages
import warnings
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Suppress specific future warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Import the clean data
data = pd.read_pickle('source\data.pkl')

data.info()

# Copy of the original dataset for feature engineering and preprocessing
data_processed = data.copy()

# Drop unnecessary columns
data_processed = data_processed.drop(['AccID', 'birth_year', 'vehicleID', 'num_veh'], axis=1)

# Convert 'day', 'month', and 'time' to integers
data_processed['day'] = data_processed['day'].astype(int)
data_processed['month'] = data_processed['month'].astype(int)
data_processed['time'] = data_processed['time'].astype(int)

# Cyclical encoding for temporal features
data_processed['day_sin'] = np.sin(2 * np.pi * data_processed['day'] / 31)  
data_processed['day_cos'] = np.cos(2 * np.pi * data_processed['day'] / 31)

data_processed['month_sin'] = np.sin(2 * np.pi * data_processed['month'] / 12)
data_processed['month_cos'] = np.cos(2 * np.pi * data_processed['month'] / 12)

data_processed['time_sin'] = np.sin(2 * np.pi * data_processed['time'] / 86340000) 
data_processed['time_cos'] = np.cos(2 * np.pi * data_processed['time'] / 86340000)

data_processed.drop(columns=['day','month','time'],inplace=True)

# Selecting features and target variable
features_dummy = ['year', 'lum', 'atm_condition', 'collision_type',
       'route_category', 'traffic_regime', 'total_number_lanes',
       'reserved_lane_code', 'longitudinal_profile', 'plan',
       'surface_condition', 'infra', 'accident_situation',
       'traffic_direction', 'vehicle_category', 'fixed_obstacle',
       'mobile_obstacle', 'initial_impact_point', 'manv', 'motor', 'seat',
       'user_category', 'gender', 'reason_travel',
       'safety_equipment1']

# These features will be standardized
features_scaler = ['lat', 'long', 'upstream_terminal_number', 'distance_upstream_terminal', 'maximum_speed', 'age']

# These features are between -1 and 1 and do not need any standardazations. 
features_temporal = ['day_sin', 'day_cos', 'month_sin', 'month_cos', 'time_sin', 'time_cos']
target = 'gravity'

X = data_processed.drop(columns=[target])
y = data_processed[target]
y = y.astype(int)

X = pd.get_dummies(X, columns=features_dummy, drop_first=True)

# stratify will split the dataset according to the distribution of the classes to compensate for imbalanced datasets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Standardization: Fit only on the training data, then apply to both train and test
scaler = StandardScaler()
X_train[features_scaler] = scaler.fit_transform(X_train[features_scaler])
X_test[features_scaler] = scaler.transform(X_test[features_scaler])

# Check the dimensions
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447670 entries, 0 to 447669
Data columns (total 39 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   AccID                       447670 non-null  object 
 1   day                         447670 non-null  object 
 2   month                       447670 non-null  object 
 3   year                        447670 non-null  object 
 4   time                        447670 non-null  object 
 5   lum                         447670 non-null  object 
 6   atm_condition               447670 non-null  object 
 7   collision_type              447670 non-null  object 
 8   lat                         447670 non-null  float64
 9   long                        447670 non-null  float64
 10  route_category              447670 non-null  object 
 11  traffic_regime              447670 non-null  object 
 12  total_number_lanes          447670 non-null  object 
 13  reserved_lane_

Apply ML v1 -------->

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize AdaBoost Classifier
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42)

# Fit the model
ada_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ada_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Generate classification report
class_report = classification_report(y_test, y_pred)

print(accuracy)
print(class_report)


0.6367413496548797
              precision    recall  f1-score   support

           1       0.69      0.80      0.74     37537
           2       0.29      0.03      0.06      2256
           3       0.50      0.36      0.42     13565
           4       0.62      0.61      0.61     36176

    accuracy                           0.64     89534
   macro avg       0.52      0.45      0.46     89534
weighted avg       0.62      0.64      0.62     89534



Apply ML v2 -------->

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Grid search to optimize for f1_weighted (since the dataset is likely imbalanced)
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1],
    'base_estimator__max_depth': [1, 2]  
}

# Initialize a base estimator for AdaBoost
base_estimator = DecisionTreeClassifier(max_depth=1, class_weight='balanced')

# Initialize the AdaBoost classifier with the base estimator
ada_clf = AdaBoostClassifier(base_estimator=base_estimator, random_state=42)

# Perform grid search with cross-validation, optimizing for weighted F1 score
grid_search = GridSearchCV(estimator=ada_clf, param_grid=param_grid, scoring='f1_weighted', cv=3, n_jobs=-1, verbose=1)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params_f1 = grid_search.best_params_
best_score_f1 = grid_search.best_score_

print(f"Best parameters: {best_params_f1}")
print(f"Best F1 score: {best_score_f1}")


Fitting 3 folds for each of 18 candidates, totalling 54 fits


In [ ]:
from sklearn.metrics import classification_report

# Train the AdaBoostClassifier with the best hyperparameters
ada_clf_best = AdaBoostClassifier(n_estimators=50, learning_rate=0.01, random_state=42)
ada_clf_best.fit(X_train, y_train)

# Predict on the test set
y_pred_best = ada_clf_best.predict(X_test)

# Generate and display the classification report
class_report_best = classification_report(y_test, y_pred_best)
class_report_best


In [ ]:
print(class_report_best
     )

In [ ]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to oversample the minority classes
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train the AdaBoostClassifier with the best hyperparameters on the resampled dataset
ada_clf_best = AdaBoostClassifier(n_estimators=200, learning_rate=1, random_state=42)
ada_clf_best.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_resampled = ada_clf_best.predict(X_test)

# Generate the classification report
class_report_resampled = classification_report(y_test, y_pred_resampled)
print(class_report_resampled)



In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Apply RandomUnderSampler to reduce the number of samples in the majority classes
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

# Train the AdaBoostClassifier with the best hyperparameters on the resampled dataset
ada_clf_best = AdaBoostClassifier(n_estimators=200, learning_rate=1, random_state=42)
ada_clf_best.fit(X_train_resampled, y_train_resampled)

# Predict on the test set
y_pred_resampled = ada_clf_best.predict(X_test)

# Generate the classification report
class_report_resampled = classification_report(y_test, y_pred_resampled)
print(class_report_resampled)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# Define a base classifier with class weights
base_clf = DecisionTreeClassifier(class_weight='balanced', max_depth=1)

# Use this base classifier in AdaBoost
ada_clf_weighted = AdaBoostClassifier(base_estimator=base_clf, n_estimators=200, learning_rate=1, random_state=42)

# Train on the original dataset
ada_clf_weighted.fit(X_train, y_train)

# Predict on the test set
y_pred_weighted = ada_clf_weighted.predict(X_test)

# Generate the classification report
class_report_weighted = classification_report(y_test, y_pred_weighted)
print(class_report_weighted)
